In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import distance_transform_edt
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

def rgb_to_labels(image, color_map):
    """
    Convert an RGB image to class labels based on a color mapping.

    Args:
        image (np.array): Input RGB image, shape (height, width, 3).
        color_map (dict): Mapping of labels to RGB colors (tuple).

    Returns:
        labels (np.array): Label array, shape (height, width).
    """
    height, width, _ = image.shape
    labels = np.zeros((height, width), dtype=np.int32)
    for label, color in color_map.items():
        mask = np.all(image == color, axis=-1)
        labels[mask] = label
    return labels

def calculate_multiclass_metrics_per_class(y_true, y_pred, num_classes):
    """
    Calculate multiclass segmentation metrics per class using a confusion matrix.

    Args:
        y_true (np.array): Ground truth labels, shape (height, width).
        y_pred (np.array): Predicted labels, shape (height, width).
        num_classes (int): Number of classes.

    Returns:
        iou_per_class (list): IoU score for each class.
        dice_per_class (list): Dice coefficient for each class.
        accuracy (float): Overall accuracy across all pixels.
        precision_per_class (list): Precision score for each class.
        recall_per_class (list): Recall score for each class.
    """
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    cm = confusion_matrix(y_true_flat, y_pred_flat, labels=range(num_classes))
    
    # Optimal mapping to align predicted clusters with ground truth classes
    row_ind, col_ind = linear_sum_assignment(-cm)
    mapped_pred = np.zeros_like(y_pred_flat)
    for i, j in zip(row_ind, col_ind):
        mapped_pred[y_pred_flat == j] = i
    
    cm_mapped = confusion_matrix(y_true_flat, mapped_pred, labels=range(num_classes))
    
    iou_per_class = []
    dice_per_class = []
    precision_per_class = []
    recall_per_class = []
    
    total_pixels = y_true_flat.size
    accuracy = np.sum(y_true_flat == mapped_pred) / total_pixels
    
    for cls in range(num_classes):
        tp = cm_mapped[cls, cls]
        fp = cm_mapped[:, cls].sum() - tp
        fn = cm_mapped[cls, :].sum() - tp
        
        iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
        iou_per_class.append(iou)
        
        dice = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
        dice_per_class.append(dice)
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        precision_per_class.append(precision)
        
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        recall_per_class.append(recall)
    
    return iou_per_class, dice_per_class, accuracy, precision_per_class, recall_per_class

def genericSegEvaluation(seg1, seg2):
    """
    Evaluate non-semantic segmentation using a confusion matrix-based metric.
    Implements the metric from https://doi.org/10.1109/TIP.2005.854491.

    Args:
        seg1 (np.array): First segmentation mask.
        seg2 (np.array): Second segmentation mask.

    Returns:
        error (float): Segmentation error in range [0, 1].
    """
    seg1_flat = seg1.flatten()
    seg2_flat = seg2.flatten()
    cm = confusion_matrix(seg1_flat, seg2_flat)
    
    # Optimal mapping to maximize matching between segmentations
    row_ind, col_ind = linear_sum_assignment(cm, maximize=True)
    quality = cm[row_ind, col_ind].sum()
    
    # Compute error as 1 - (matched pixels / total pixels)
    error = 1 - quality / (seg1_flat.size - 1)
    return error

def extract_glcm_features(image):
    """
    Extract GLCM-based texture features from a grayscale image.

    Args:
        image (np.array): Grayscale image, shape (height, width).

    Returns:
        contrast (np.array): Contrast feature, same shape as input.
        homogeneity (np.array): Homogeneity feature, same shape as input.
        energy (np.array): Energy feature, same shape as input.
        correlation (np.array): Correlation feature, same shape as input.
    """
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return (
        np.full(image.shape, contrast),
        np.full(image.shape, homogeneity),
        np.full(image.shape, energy),
        np.full(image.shape, correlation)
    )

def extract_lbp_features(image):
    """
    Extract Local Binary Pattern (LBP) features from a grayscale image.

    Args:
        image (np.array): Grayscale image, shape (height, width).

    Returns:
        lbp (np.array): LBP feature map, same shape as input.
        lbp_copy (np.array): Duplicate of LBP feature map (placeholder).
    """
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    return lbp, lbp  # Duplicate return as placeholder

def apply_gabor_filter(image):
    """
    Apply a Gabor filter to a grayscale image for texture analysis.

    Args:
        image (np.array): Grayscale image, shape (height, width).

    Returns:
        filtered (np.array): Filtered image, same shape as input.
    """
    ksize = 31
    theta = np.pi / 4
    sigma = 5.0
    lambd = 10.0
    gamma = 0.5
    gabor = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma)
    filtered = cv2.filter2D(image, cv2.CV_64F, gabor)
    return filtered

def extract_features(image):
    """
    Extract a set of features from an RGB image for segmentation.

    Args:
        image (np.array): Input RGB image, shape (height, width, 3).

    Returns:
        features (np.array): Feature matrix, shape (height * width, 13).
    """
    height, width, channels = image.shape
    features = np.zeros((height, width, 13))

    # Normalize image to [0, 1]
    image_norm = image.astype(np.float32) / 255.0
    features[:, :, :3] = image_norm  # RGB channels

    # Local mean
    mean_local = cv2.blur(image_norm, (11, 11))
    features[:, :, 3] = cv2.cvtColor(mean_local, cv2.COLOR_BGR2GRAY)

    # Edge detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 100, 200)
    features[:, :, 4] = edges / 255.0

    # Distance transform from edges
    dist_transform = distance_transform_edt(1 - (edges / 255.0))
    features[:, :, 5] = dist_transform / np.max(dist_transform) if np.max(dist_transform) > 0 else dist_transform

    # GLCM features
    contrast, homogeneity, energy, correlation = extract_glcm_features(gray_image)
    features[:, :, 6] = contrast
    features[:, :, 7] = homogeneity
    features[:, :, 8] = energy
    features[:, :, 9] = correlation

    # LBP features
    lbp_hist = extract_lbp_features(gray_image)
    features[:, :, 10] = lbp_hist[0] / np.max(lbp_hist[0]) if np.max(lbp_hist[0]) > 0 else lbp_hist[0]
    features[:, :, 11] = lbp_hist[1] / np.max(lbp_hist[1]) if np.max(lbp_hist[1]) > 0 else lbp_hist[1]

    # Gabor filter
    gabor_features = apply_gabor_filter(gray_image)
    features[:, :, 12] = gabor_features / np.max(np.abs(gabor_features)) if np.max(np.abs(gabor_features)) > 0 else gabor_features

    return features.reshape((-1, 13))

In [ ]:
# Directory paths
test_input_path = "../../experiments_data/test_input"
test_target_path = "../../experiments_data/test_target"
output_path = "test_results_kmeans_multi"

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Color mapping for 5 classes (in BGR)
color_map = {
    0: (0, 156, 255),   # Orange
    1: (0, 255, 0),     # Green
    2: (77, 255, 255),  # Light cyan
    3: (255, 0, 0),     # Blue
    4: (255, 0, 255)    # Magenta
}

def process_multiclass_segmentation():
    """
    Process test images for 5-class segmentation using K-Means clustering.
    Saves segmented images and comparison visualizations.

    Args:
        None

    Returns:
        None
    """
    for filename in os.listdir(test_input_path):
        if filename.endswith(".tif"):
            # Construct file paths
            test_image_path = os.path.join(test_input_path, filename)
            test_labels_path = os.path.join(test_target_path, filename.replace(".tif", ".png"))
            output_image_path = os.path.join(output_path, f"resultado_kmeans_5_{filename.replace('.tif', '.png')}")
            comparison_path = os.path.join(output_path, f"comparacao_kmeans_5_{filename.replace('.tif', '.png')}")

            # Load input and ground truth images
            test_image = cv2.imread(test_image_path)
            test_labels_rgb = cv2.imread(test_labels_path)

            if test_image is None or test_labels_rgb is None:
                print(f"Error loading {filename} or its corresponding mask!")
                continue

            # Extract features for clustering
            features = extract_features(test_image)

            # Normalize features
            scaler = StandardScaler()
            features_normalized = scaler.fit_transform(features)

            # Apply K-Means with 5 clusters
            kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
            predicted_labels = kmeans.fit_predict(features_normalized)
            predicted_labels = predicted_labels.reshape(test_image.shape[:2])

            # Create segmented image with class colors
            segmented_image = np.zeros((test_image.shape[0], test_image.shape[1], 3), dtype=np.uint8)
            for label in range(5):
                segmented_image[predicted_labels == label] = color_map[label]

            # Save segmented image
            cv2.imwrite(output_image_path, segmented_image)
            print(f"Result saved at: {output_image_path}")

            # Generate and save comparison visualization
            plt.figure(figsize=(18, 6))
            plt.subplot(1, 3, 1)
            plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
            plt.title("Original Image")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))
            plt.title("Segmentation (K-Means 5 Classes)")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.imshow(cv2.cvtColor(test_labels_rgb, cv2.COLOR_BGR2RGB))
            plt.title("Target Image")
            plt.axis("off")

            plt.savefig(comparison_path)
            plt.close()
            print(f"Comparison saved at: {comparison_path}")

    print("Processing completed!")

process_multiclass_segmentation()

Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID1[x=40320,y=25984,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID1[x=40320,y=25984,w=1024,h=1024].png


c:\Users\emagn\Documents\GitHub\KidneySegmentation\MyEnv\Lib\site-packages\sklearn\preprocessing\_data.py:1020: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID3[x=19712,y=25088,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID3[x=19712,y=25088,w=1024,h=1024].png


c:\Users\emagn\Documents\GitHub\KidneySegmentation\MyEnv\Lib\site-packages\sklearn\preprocessing\_data.py:1020: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID4[x=10752,y=11648,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID4[x=10752,y=11648,w=1024,h=1024].png


...

Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID4[x=11648,y=12544,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID4[x=11648,y=12544,w=1024,h=1024].png
Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID4[x=5376,y=7168,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID4[x=5376,y=7168,w=1024,h=1024].png
Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID5[x=2688,y=14336,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID5[x=2688,y=14336,w=1024,h=1024].png


c:\Users\emagn\Documents\GitHub\KidneySegmentation\MyEnv\Lib\site-packages\sklearn\preprocessing\_data.py:1020: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID5[x=3584,y=17920,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID5[x=3584,y=17920,w=1024,h=1024].png


c:\Users\emagn\Documents\GitHub\KidneySegmentation\MyEnv\Lib\site-packages\sklearn\preprocessing\_data.py:1020: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


Result saved at: test_results_kmeans_5classes_norm/resultado_kmeans_5_ID7[x=5376,y=29568,w=1024,h=1024].png
Comparison saved at: test_results_kmeans_5classes_norm/comparacao_kmeans_5_ID7[x=5376,y=29568,w=1024,h=1024].png
Processing completed!


In [ ]:
# Directory paths
test_target_path = "../../experiments_data/test_target"
output_path = "test_results_kmeans_multi"
metrics_file = os.path.join(output_path, "metrics.txt")

# Ground truth color mapping (assumes color_map is defined)
ground_truth_color_map = color_map.copy()

def evaluate_multiclass_segmentation():
    """
    Evaluate 5-class segmentation results and compute metrics.
    Saves IoU, Dice, Precision, Recall, and accuracy metrics to a file.

    Args:
        None

    Returns:
        None
    """
    # Initialize lists to store metrics for each class
    num_classes = 5
    iou_all_images = [[] for _ in range(num_classes)]
    dice_all_images = [[] for _ in range(num_classes)]
    precision_all_images = [[] for _ in range(num_classes)]
    recall_all_images = [[] for _ in range(num_classes)]
    accuracy_all_images = []
    generic_seg_errors = []

    # Process saved segmentation results
    for filename in os.listdir(output_path):
        if filename.startswith("resultado_kmeans_5_") and filename.endswith(".png"):
            # Construct file paths
            original_filename = filename.replace("resultado_kmeans_5_", "").replace(".png", ".tif")
            test_labels_path = os.path.join(test_target_path, original_filename.replace(".tif", ".png"))
            segmented_image_path = os.path.join(output_path, filename)

            # Load ground truth and segmented images
            test_labels_rgb = cv2.imread(test_labels_path)
            segmented_image = cv2.imread(segmented_image_path)

            if test_labels_rgb is None or segmented_image is None:
                print(f"Error loading {original_filename} or its segmented result!")
                continue

            # Convert images to class labels
            y_true = rgb_to_labels(test_labels_rgb, ground_truth_color_map)
            y_pred = rgb_to_labels(segmented_image, color_map)

            # Compute per-class metrics
            iou_per_class, dice_per_class, accuracy, precision_per_class, recall_per_class = calculate_multiclass_metrics_per_class(y_true, y_pred, num_classes)

            # Compute segmentation error
            error = genericSegEvaluation(y_true, y_pred)

            # Store metrics
            for cls in range(num_classes):
                iou_all_images[cls].append(iou_per_class[cls])
                dice_all_images[cls].append(dice_per_class[cls])
                precision_all_images[cls].append(precision_per_class[cls])
                recall_all_images[cls].append(recall_per_class[cls])
            accuracy_all_images.append(accuracy)
            generic_seg_errors.append(error)

    # Calculate average metrics across images
    mean_iou_per_class = [np.mean(iou_all_images[cls]) for cls in range(num_classes)]
    mean_dice_per_class = [np.mean(dice_all_images[cls]) for cls in range(num_classes)]
    mean_precision_per_class = [np.mean(precision_all_images[cls]) for cls in range(num_classes)]
    mean_recall_per_class = [np.mean(recall_all_images[cls]) for cls in range(num_classes)]
    mean_accuracy = np.mean(accuracy_all_images)
    mean_generic_seg_error = np.mean(generic_seg_errors)

    # Write metrics to file
    class_names = ["Orange", "Green", "Light Yellow", "Blue", "Magenta"]
    with open(metrics_file, 'w') as f:
        f.write("Metric Results (K-Means 5 Classes)\n")
        f.write("==================================\n")
        
        for cls in range(num_classes):
            f.write(f"Class: {class_names[cls]}\n")
            f.write(f"Mean IoU: {mean_iou_per_class[cls]:.4f}\n")
            f.write(f"Mean Dice: {mean_dice_per_class[cls]:.4f}\n")
            f.write(f"Mean Precision: {mean_precision_per_class[cls]:.4f}\n")
            f.write(f"Mean Recall: {mean_recall_per_class[cls]:.4f}\n")
            f.write("----------------------------------\n")
        
        f.write(f"Mean Accuracy: {mean_accuracy:.4f}\n")
        f.write(f"Mean genericSegEvaluation Error: {mean_generic_seg_error:.4f}\n")

    print(f"Metrics saved at: {metrics_file}")

evaluate_multiclass_segmentation()

Metrics saved at: test_results_kmeans_5classes_norm/metrics.txt
